# **Chatbot Implementation Using Langchain**

### **Install Required Libraries**

In [12]:
!pip -q install langchain langchain-community langchain-openai openai pypdf sentence_transformers tiktoken faiss-cpu unstructured

In [13]:
!pip install numpy==1.24.4
!pip install nltk==3.9.1

  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.9.0 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.4 which is incompatible.


### **Load Required Libraries**

In [20]:
# Import the sys module for interacting with the interpreter and command-line arguments
import sys

# Import the os module for interacting with the operating system (e.g., file paths, environment variables)
import os

# Import the PyTorch library for tensor operations and deep learning
import torch

# Import the textwrap module for formatting text (e.g., wrapping long lines)
import textwrap

# Import UnstructuredURLLoader for loading documents from unstructured URLs
from langchain.document_loaders import UnstructuredURLLoader

# Import CharacterTextSplitter to split text into smaller chunks based on character limits
from langchain.text_splitter import CharacterTextSplitter

# Import OpenAIEmbeddings for generating embeddings using OpenAI models
from langchain_openai import OpenAIEmbeddings

# Import ChatOpenAI to enable chat functionality with OpenAI's language models
from langchain_openai import ChatOpenAI

# Import FAISS for efficient similarity search and clustering of dense vectors
from langchain.vectorstores import FAISS

# Import RetrievalQAWithSourcesChain for building a question-answering chain that retrieves sources
from langchain.chains import RetrievalQAWithSourcesChain

# Import HuggingFaceEmbeddings for generating embeddings using Hugging Face models
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
# Import the Natural Language Toolkit (nltk) library for natural language processing tasks
import nltk

# Download the 'punkt' tokenizer model, which is used for splitting text into sentences or words
nltk.download('punkt')

# Download the 'averaged_perceptron_tagger' model, which is used for part-of-speech tagging
nltk.download('averaged_perceptron_tagger')

# Ensure the punkt tokenizer is found
nltk.data.find('tokenizers/punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


FileSystemPathPointer('/root/nltk_data/tokenizers/punkt')

### **Set Up API Key**

In [3]:
os.environ['OPENAI_API_KEY'] = "API_KEY"

### **Paste the URLs and extract the data from these URLs**

In [4]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'
]


### **Load Website Data**

In [5]:
# Create an instance of UnstructuredURLLoader to load documents from a list of URLs
loaders = UnstructuredURLLoader(
    urls=URLs  # Provide the list of URLs from which to load unstructured data
)

# Error handling for URL loading
try:
    # Load the data from the specified URLs using the loader
    data = loaders.load()  # This method retrieves and processes the content from the URLs
    print("Data loaded successfully.")
except Exception as e:
    print(f"Error loading data: {e}")

Data loaded successfully.


In [7]:
# data #uncomment to see data

In [8]:
len(data) # 4 because we loaded 4 web links

4

## **Create Chunks of Data**

In [9]:
# Create an instance of CharacterTextSplitter to split text into manageable chunks
text_splitter = CharacterTextSplitter(
    # The character used to separate the text into chunks
    separator='\n',  # Split the text at newline characters
    # The maximum size of each chunk in characters
    chunk_size=1000,  # Each chunk will contain up to 1000 characters
    # The number of overlapping characters between consecutive chunks
    chunk_overlap=200  # Each chunk will overlap with the next by 200 characters
)

In [10]:
# Use the text_splitter instance to split the documents in 'data' into smaller chunks
text_chunks = text_splitter.split_documents(data)

# Explanation:
# - data: This is expected to be a collection of documents (e.g., a list of strings or text objects) that you want to split into smaller, manageable pieces.
# - text_chunks: This variable will hold the resulting list of text chunks after the split operation.
# The split_documents method will apply the specified chunking logic (separator, chunk_size, and chunk_overlap) defined in the text_splitter instance.

In [12]:
# Get the number of text chunks created by the text_splitter
number_of_chunks = len(text_chunks)
print(f"Number of chunks: {number_of_chunks}")
# Explanation:
# - text_chunks: This variable contains the list of text chunks obtained from the previous split operation.
# - len(text_chunks): This function returns the total number of chunks in the list.
# The result is stored in 'number_of_chunks', which indicates how many individual text chunks were generated from the original documents.

Number of chunks: 86


### **Let's Check First Chunk**

In [13]:
text_chunks[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhanc

## **Load OpenAI Embedding Model**

In [15]:
embeddings = OpenAIEmbeddings()

In [18]:
# Generate an embedding for the query string "Hello world"
query_result = embeddings.embed_query("Hello world")

# Get the length of the resulting embedding
embedding_length = len(query_result)
print(f"Embedding length: {embedding_length}")
# Explanation:
# - embeddings: This is an instance of an embedding model that transforms text into a numerical representation (embedding).
# - embed_query("Hello world"): This method takes the input query (in this case, "Hello world") and computes its embedding.
# - query_result: This variable holds the resulting embedding, which is typically a list or array of numerical values representing the query.
# - len(query_result): This function returns the total number of elements in the embedding.
# The result is stored in 'embedding_length', which indicates the dimensionality of the embedding for the given query.

Embedding length: 1536


## **Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [19]:
# Create a FAISS vector store from the text chunks and the embeddings model
vectorstore = FAISS.from_documents(text_chunks, embeddings)

# Explanation:
# - FAISS: This refers to the Facebook AI Similarity Search library, which is used for efficient similarity search and clustering of dense vectors.
# - from_documents(text_chunks, embeddings): This method initializes a FAISS vector store by processing the provided text chunks.
#   - text_chunks: This is a list of text chunks that were previously split from the original documents.
#   - embeddings: This is the embedding model used to convert the text chunks into dense vector representations.
# - vectorstore: This variable holds the resulting FAISS vector store, which allows for efficient retrieval and similarity searches based on the embeddings of the text chunks.

## **Create a Large Language Model (LLM) Wrapper**

In [21]:
# Create an instance of the ChatOpenAI model for conversational AI tasks
llm = ChatOpenAI()

# Explanation:
# - ChatOpenAI: This refers to a class or function that initializes a language model designed for chat-based interactions, typically leveraging OpenAI's GPT architecture.
# - llm: This variable holds the instance of the ChatOpenAI model, which can be used to generate responses, engage in dialogues, and perform various natural language processing tasks.

In [24]:
llm.invoke("Please provide a concise summary of the Book Harry Potter")

AIMessage(content='Harry Potter is a popular fantasy book series written by J.K. Rowling that follows the life of a young wizard named Harry Potter as he attends Hogwarts School of Witchcraft and Wizardry. Throughout the series, Harry and his friends Ron and Hermione battle the dark wizard Lord Voldemort and his followers, the Death Eaters. The story explores themes of friendship, love, courage, and the power of good over evil.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 17, 'total_tokens': 99, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-03bf28e7-ae4d-4218-9697-7c435d23c3fc-0', usage_metadata={'input_tokens': 17, 'output_tokens': 82, 'total_tokens': 99, 'input_token_details': {'cache_read': 0}, 'outpu

## **Initialize the Retrieval QA with Source Chain**

In [25]:
# Create a RetrievalQAWithSourcesChain that combines the language model with a retriever
chain = RetrievalQAWithSourcesChain.from_llm(
    llm=llm,  # The ChatOpenAI instance used for generating responses
    retriever=vectorstore.as_retriever()  # Convert the FAISS vector store to a retriever
)

# Explanation:
# - RetrievalQAWithSourcesChain: This is a class that facilitates a question-answering system with retrieval capabilities, allowing it to fetch relevant documents based on a query before generating a response.
# - from_llm(): This method initializes the chain using the specified language model and retriever.
#   - llm: The ChatOpenAI instance that will be used to generate answers based on the retrieved documents.
#   - retriever: This converts the FAISS vector store into a retriever interface, enabling the chain to search for relevant text chunks based on input queries.
# - chain: This variable holds the instance of the RetrievalQAWithSourcesChain, which can process questions, retrieve documents, and generate answers that include sources.

In [27]:
result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)

<ipython-input-27-6dc5d9fb268a>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)
/usr/local/lib/python3.10/dist-packages/langchain_openai/chat_models/base.py:356: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [28]:
result['answer']

'Vicuna is considered to be very good, achieving high quality comparable to ChatGPT, OpenAI ChatGPT, and Google Bard, and outperforming models like LLaMA and Stanford Alpaca in over 90% of cases.\n'